# Extract

In [1]:
import gzip             #importo la librería para leer los archivos gzip, formato de compresion en el que vinieron los archivos
import pandas as pd     #importo librería pandas que vamos a usar para el data frame
import ast              #importo librería ast que me ayuda a leer la gramatica abstracta

### Steamgames

In [2]:
#si bien se podría descomprimir el archivo y directamente leerlo de pandas, dado que futuras actualziaciones pueden venir como gzip, es preferible trabajarlos desde ese formato
with gzip.open('Datos_sin_procesar/steam_games.json.gz', 'rb') as f:
    df_steamgames=pd.read_json(f, lines=True)

### User Items

In [3]:
#comienzo con una lista para poder insertar cada línea del archivo
data=[]
with gzip.open('Datos_sin_procesar/users_items.json.gz', 'rb') as f:
    for line in f:
        #cargar la información usando ast, porque si se usa json se dan errores por sintacis de gramatica en el uso de comillas
        data.append(ast.literal_eval(line.decode('utf-8')))

# Convierto la lista a DataFrame
df_users_items = pd.DataFrame(data)

### User Reviews

In [21]:
#comienzo con una lista para poder insertar cada línea del archivo
data=[]
with gzip.open('Datos_sin_procesar/user_reviews.json.gz', 'rb') as f:
    for line in f:
        #cargar la información usando ast, porque si se usa json se dan errores por sintacis de gramatica en el uso de comillas
        data.append(ast.literal_eval(line.decode('utf-8')))

# Convierto la lista a DataFrame
df_user_reviews = pd.DataFrame(data)

# Transorm

### Steamgames

In [4]:
#se eliminan los valores con todos nulos
df_steamgames.dropna(inplace= True, how='all')

In [5]:
#verifico algún id que esté nulo.
df_steamgames[df_steamgames['id'].isnull()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [6]:
#se verifica sin con la url podemos obetener el id del juego
df_steamgames[['url','id']]
#se concluye que sí, y se pasa a asignar ese valor a los faltantes

,url,id
88310,http://store.steampowered.com/app/761140/Lost_...,761140.0
88311,http://store.steampowered.com/app/643980/Ironb...,643980.0
88312,http://store.steampowered.com/app/670290/Real_...,670290.0
88313,http://store.steampowered.com/app/767400/2222/,767400.0
88314,http://store.steampowered.com/app/773570/Log_C...,773570.0
...,...,...
120440,http://store.steampowered.com/app/773640/Colon...,773640.0
120441,http://store.steampowered.com/app/733530/LOGis...,733530.0
120442,http://store.steampowered.com/app/610660/Russi...,610660.0
120443,http://store.steampowered.com/app/658870/EXIT_...,658870.0


In [7]:
#se importa la libreria re para poder trabajar sobre la extracción del ID de la url, y se genera una función para este propósito
import re

def extraer_numero_url(url):
    if url is None:
        return None
    else:
        match = re.search(r'(\d+)', url)
        if match:
            return match.group(1)
        else:
            return None

#aplicamos la funcion a una columna auxiliar
df_steamgames['id_aux']=df_steamgames['url'].apply(extraer_numero_url)
#aplicamos la columna auxiliar a los valores nulos
df_steamgames['id'][df_steamgames['id'].isnull()]=df_steamgames['id_aux']
#eliminamos los id nulos
df_steamgames.dropna(subset=['id'], inplace=True)
#eliminamos la columna auxiliar
df_steamgames.drop(columns=['id_aux'], inplace=True)


C:\Users\feder\AppData\Local\Temp\ipykernel_16760\4277996820.py:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_steamgames['id'][df_steamgames['id'].isnull()]=df_steamgames['id_aux']
C:\Users\feder\AppData\Local\Temp\ipykernel_16760\427

La columna genres tiene listas con los generos a los que puede pertenecer el juego. Para un mejor procesamiento a posteriori de los datos, se transforma esa columna en varias columnas dummies con cada uno de esos valores.
También se tiene la columna tags, pero que tiene muchos más valores. Algunos de ellos son las categorías, y también otros tipos de etiquetas. Se arma un dataframe para rellenar los valores faltantes de etiquetas.

In [8]:
#Lleno el title nulo con el app_name
df_steamgames.loc[df_steamgames['title'].isnull(), 'title'] = df_steamgames['app_name']
#cambio los nombres de columas que se usan más adelante
df_steamgames = df_steamgames.rename(columns={'id': 'item_id'})
df_steamgames = df_steamgames.rename(columns={'title': 'item_title'})

In [9]:
#Lleno la columna tags nula con la columna genres. Esto se utiliza luego en el algotirmo de ML
df_steamgames.loc[df_steamgames['tags'].isnull(), 'tags'] = df_steamgames['genres']

In [10]:
#primero llenamos los nulos con valor [], para su posterior procesamiento

# Define una función para reemplazar los valores nulos con una lista vacía
def reemplazonulo(lst):
    return lst if isinstance(lst, list) else []

# Aplica la función a la columna con listas
df_steamgames['genres'] = df_steamgames['genres'].apply(reemplazonulo)
df_steamgames['tags'] = df_steamgames['tags'].apply(reemplazonulo)

In [11]:
#Separo la columna de generos
df_genres = df_steamgames['genres'].apply(lambda x: pd.Series({val: 1 for val in x}))
#lleno los nulos con 0
df_genres.fillna(0, inplace=True)
#Se renombran columnas que tienen mal los nombres
df_genres.rename(columns={'Design &amp; Illustration':'Design & Illustration','Animation &amp; Modeling':'Animation & Modeling'}, inplace= True)
#armo una lista con los generos
list_Genres=df_genres.columns.to_list()


In [12]:
#Se genera el dataframe de de tags
df_tags = df_steamgames['tags'].apply(lambda x: pd.Series({val: 1 for val in x}))
df_tags.fillna(0, inplace=True)
#idem caso anterior con nombres mal importados
df_tags.rename(columns={'Design &amp; Illustration':'Design & Illustration','Animation &amp; Modeling':'Animation & Modeling'}, inplace= True)

In [13]:
#generamos un dataframe que usaremos luego en el algoritmo de ML
df_tags_for_ML=pd.concat([df_steamgames[['item_title', 'item_id']],df_tags], axis=1)
#cambiamos de formato sus columnas para que ocupen menos espacio
columnas=df_tags_for_ML.columns.tolist()
columnas.remove('item_title')
for i in columnas:
    df_tags_for_ML[i]= df_tags_for_ML[i].astype(int)
#se exporta el dataframe, y se elimina
df_tags_for_ML.to_csv('Datos_procesados/df_tags_for_ML.csv')
#se elimina para que no ocupe más espacio
del df_tags_for_ML

In [14]:
genres=df_genres.columns.tolist()
#completamos las categorías que son nulas en la columna de categoria pero si fueron etiquetadas usando los tags
for i in genres:
    for index, row in df_genres.iterrows():
        if row[i] == 0:
            df_genres.at[index, i] = df_tags.at[index, i]
df_steamgames=pd.concat([df_steamgames,df_genres], axis=1)

#eliminamos los dataframes que ya no usamos
del df_genres
del df_tags

In [15]:
#Pongo el año de lanzamiento
df_steamgames['release_date'] = pd.to_datetime(df_steamgames['release_date'], errors='coerce')
df_steamgames.loc[~df_steamgames['release_date'].notnull(), 'release_date'] = None
df_steamgames['release_year'] = df_steamgames['release_date'].dt.year

In [16]:
#cambio los tipos de datos para ocupar menos espacio
df_steamgames['release_year'] = df_steamgames['release_year'].fillna(0)
df_steamgames['release_year'] = df_steamgames['release_year'].astype(int)
for i in genres:
    df_steamgames[i] = df_steamgames[i].fillna(0)
    df_steamgames[i] = df_steamgames[i].astype(int)
df_steamgames['item_id']=df_steamgames['item_id'].astype(int)

In [ ]:
#elimino lo que no sirve
df_steamgames.drop(['publisher','app_name','genres', 'tags' ,'url','release_date','reviews_url','specs','early_access',
                                 'developer'],axis=1,inplace=True)

### User Items

Se posee en la columna items listas con diccionarios. Cada diccionario tiene 4 atributos. Se genera una fila para cada diccionario, y 4 columnas posteriores a partir del diccionario

In [17]:
#primero se inserta una fila por cada diccionario de la columna item
df_users_items=df_users_items.explode('items')
df_users_items = df_users_items.reset_index(drop=True)

In [18]:
#luego se arma un dataframe nuevo a partir de los diccionarios de item, con una columna por cada atributo
# el metodo normal sería este, pero tarda mucho, con lo cual buscamos extraer columna por columna df_users_items_items=df_users_items['items'].apply(pd.Series)

def obtener_elemento(diccionario, clave_busqueda):
    if isinstance(diccionario, dict):
        return diccionario.get(clave_busqueda)
    else:
        return diccionario

df_users_items['item_id'] = df_users_items['items'].apply(lambda x: obtener_elemento(x, 'item_id'))
df_users_items['item_name'] = df_users_items['items'].apply(lambda x: obtener_elemento(x, 'item_name'))
df_users_items['playtime_forever'] = df_users_items['items'].apply(lambda x: obtener_elemento(x, 'playtime_forever'))
df_users_items['playtime_2weeks'] = df_users_items['items'].apply(lambda x: obtener_elemento(x, 'playtime_2weeks'))

In [19]:
#Saco las columnas items que ya no no utiles
df_users_items.drop(['steam_id','user_url','items', 'items_count'], axis=1,inplace= True)
#eliminamos las filas que no tienen valor de juego
df_users_items = df_users_items.dropna(subset=['item_id', 'item_name'], how='all')

### User Reviews

Cada objeto de la columna review es una lista que tiene varios valores de diccionario, de 7 atributos 'funny', 'posted',   'last_edited', 'item_id', 'helpful', 'recommend', 'review'. Se debe transformar cada diccionario en una fila, y cada uno de estos 7 atriburos en una columna, de manera tal que el dataframe refleje la realidad de datos que tenemos

In [22]:
#paso cada lista a una fila, aplicando metodo explode
df_user_reviews=df_user_reviews.explode('reviews')
#paso cada atributo del diccionario a una columna nueva, aplicando metodo apply
df_user_reviews_exp=df_user_reviews['reviews'].apply(pd.Series)
df_user_reviews.reset_index()

,index,user_id,user_url,reviews
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...
59328,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [23]:
#junto el dataframe original y las nuevas columnas
df_user_reviews=pd.concat([df_user_reviews, df_user_reviews_exp], axis=1)
#elimino la columna original
df_user_reviews.drop(['reviews','user_url', 'funny', 'last_edited', 
                      'helpful', 0], axis=1, inplace=True)

In [24]:
from textblob import TextBlob
df_user_reviews['review'] = df_user_reviews['review'].astype(str)
df_user_reviews['polarity'] = df_user_reviews['review'].apply(lambda text: TextBlob(text).sentiment.polarity)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(df_user_reviews['polarity'], bins=100)

In [25]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['polarity'].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))
df_user_reviews.drop(['review','polarity'], axis=1, inplace=True)

In [26]:
#sacar el año de la review
df_user_reviews['date'] = df_user_reviews['posted'].str.extract(r'(\w+\s\d+(?:,\s\d{4})?)')
df_user_reviews['date'] = pd.to_datetime(df_user_reviews['date'], format='%B %d, %Y', errors='coerce')
df_user_reviews['year'] = df_user_reviews['date'].dt.year
# Convertir los NaN a 0 en la columna 'year'
df_user_reviews['year'] = df_user_reviews['year'].fillna(0)
# Convertir la columna 'year' a tipo entero
df_user_reviews['year'] = df_user_reviews['year'].astype(int)

In [27]:
#dado que el ultimo año de las reseñas que tienen fecha es el 2015, parece que fueron realizadas en el año 2016,
# y cuando se realizaban en ese año no se ponía la fecha. Por eso relleno el valor de los años de reseña faltante 
# con el mayor entre ese año, y el año en que se lanzó el juego
df_steamgames.reset_index()

# Definir una función personalizada para aplicar la lógica deseada
def fill_year(row):
    if row['year'] == 0:
        # Obtener las filas de df_steamgames que cumplen con la condición
        steamgame_rows = df_steamgames[df_steamgames['item_id'] == row['item_id']]
        
        if not steamgame_rows.empty:
            # Si se encontraron filas, obtener el año de la primera fila y verificar si es mayor a 2016
            release_year = steamgame_rows['release_year'].iloc[0]
            return max(release_year, 2016)
        else:
            # Si no se encontraron filas, retornar 2016
            return 2016
    else:
        return row['year']

# Aplicar la función personalizada a la columna 'year' utilizando apply
df_user_reviews['year'] = df_user_reviews.apply(fill_year, axis=1)

# Convertir la columna 'year' a tipo entero
df_user_reviews['year'] = df_user_reviews['year'].astype(int)

In [28]:
#Agrupo lo que me sirve
df_user_item_played_year = df_user_reviews.groupby(['item_id', 'user_id', 'year']).size().reset_index(name='count')
df_user_item_played_year.drop('count', axis=1, inplace=True)
df_user_item_played_year['item_id']=df_user_item_played_year['item_id'].astype(int)

In [55]:
df_user_reviews

,user_id,posted,item_id,recommend,sentiment_analysis,date,year
0,76561197970982479,"Posted November 5, 2011.",1250,True,2,2011-11-05,2011
0,76561197970982479,"Posted July 15, 2011.",22200,True,2,2011-07-15,2011
0,76561197970982479,"Posted April 21, 2011.",43110,True,2,2011-04-21,2011
1,js41637,"Posted June 24, 2014.",251610,True,2,2014-06-24,2014
1,js41637,"Posted September 8, 2013.",227300,True,0,2013-09-08,2013
...,...,...,...,...,...,...,...
25797,76561198312638244,Posted July 10.,70,True,2,NaT,2016
25797,76561198312638244,Posted July 8.,362890,True,2,NaT,2016
25798,LydiaMorley,Posted July 3.,273110,True,2,NaT,2016
25798,LydiaMorley,Posted July 20.,730,True,2,NaT,2016


In [58]:
#se generan dos dataframes para su posterior uso, uno para usuarios y otro para juegos, para su uso en el EDA
#agrupado por juego
df_reviews_by_game=df_user_reviews.groupby('item_id').agg({
    'recommend': 'mean',
    'sentiment_analysis': 'mean',
    'user_id': 'count'
}).reset_index()
df_reviews_by_game.columns = ['item_id', 'mean_recommend', 'mean_sentiment_analysis', 'total_reviews']
df_reviews_by_game.set_index('item_id')

#agrupado por jugador
df_reviews_by_player=df_user_reviews.groupby('user_id').agg({
    'recommend': 'mean',
    'sentiment_analysis': 'mean',
    'item_id': 'count'
}).reset_index()
df_reviews_by_player.columns = ['user_id', 'mean_recommend', 'mean_sentiment_analysis', 'total_reviews']
df_reviews_by_player.set_index('user_id')

,mean_recommend,mean_sentiment_analysis,total_reviews
user_id,,,
--000--,1.0,1.000000,1
--ace--,1.0,2.000000,2
--ionex--,1.0,2.000000,2
-2SV-vuLB-Kg,1.0,2.000000,5
-Azsael-,1.0,0.000000,1
...,...,...,...
zwanzigdrei,1.0,2.000000,1
zy0705,1.0,2.000000,1
zynxgameth,1.0,2.000000,1


In [60]:
df_user_reviews

,user_id,posted,item_id,recommend,sentiment_analysis,date,year
0,76561197970982479,"Posted November 5, 2011.",1250,True,2,2011-11-05,2011
0,76561197970982479,"Posted July 15, 2011.",22200,True,2,2011-07-15,2011
0,76561197970982479,"Posted April 21, 2011.",43110,True,2,2011-04-21,2011
1,js41637,"Posted June 24, 2014.",251610,True,2,2014-06-24,2014
1,js41637,"Posted September 8, 2013.",227300,True,0,2013-09-08,2013
...,...,...,...,...,...,...,...
25797,76561198312638244,Posted July 10.,70,True,2,NaT,2016
25797,76561198312638244,Posted July 8.,362890,True,2,NaT,2016
25798,LydiaMorley,Posted July 3.,273110,True,2,NaT,2016
25798,LydiaMorley,Posted July 20.,730,True,2,NaT,2016


# Transform combinations

#### df_genres_detail

In [29]:
#Agrego las horas totales de cada juego
#primero armo un dataframe que es la suma total de horas jugadas de cada juego
df_hoursxgame = df_users_items.groupby(by=['item_id', 'item_name'])['playtime_forever'].sum().reset_index()
#cambio el tipo de item_id a integer
df_hoursxgame['item_id']=df_hoursxgame['item_id'].astype(int)
#agrego las horas al dataframe de steamgames
df_steamgames = df_steamgames.merge(df_hoursxgame[['item_id', 'playtime_forever']], on='item_id', how='left')

In [30]:
#genero una lista, que luego paso a dataframe, en la que busco el año con el maximo de horas jugadas
genres_RY_list=[]
#voy iterando por cada género
for i in genres:
    # genero un dataframe con cada año de lanzamiento y las horas jugadas, un agrupado del dataframe steamgames
    df_append = df_steamgames[df_steamgames[i] == 1].groupby(by=['release_year'])['playtime_forever'].sum().reset_index()
    # Encuentro el año con la máxima cantidad de horas jugadas
    max_hours_year = df_append.loc[df_append['playtime_forever'].idxmax(), 'release_year']
    # Agrego los datos a la lista de géneros y años de máximas horas jugadas
    genres_RY_list.append({'Genre': i, 'RYMaxHours': max_hours_year})
df_genres=pd.DataFrame(genres_RY_list)
df_genres['Genre']=df_genres['Genre'].astype(str)

In [31]:
#cambio el tipo de valor de item id
df_users_items['item_id']=df_users_items['item_id'].astype(int)
#Incluyo los generos en el dataframe de los usuarios
df_users_items = df_users_items.merge(df_steamgames, on='item_id', how='left')
#cambio nomnbre y elimino columnas duplicadas
df_users_items.rename(columns={'playtime_forever_x':'playtime_forever'}, inplace=True)
df_users_items.drop('playtime_forever_y', axis=1, inplace=True)
#convierto a enteros las columnas de generos para que ocupe menos memoria
for i in genres:
    df_users_items[i] = df_users_items[i].fillna(0).astype(int)

In [32]:
## JUGADOR QUE MAS HORAS JUGO

#genero una lista, que luego paso a dataframe, en la que busco el jugador con el maximo de horas jugadas
genres_MaxPlayer_list=[]
#voy iterando por cada género
for index, row in df_genres.iterrows():
    genero = row['Genre']
    # genero un dataframe con cada jugador y las horas jugadas de ese genero, un agrupado del dataframe df_users_items
    df_append = df_users_items[df_users_items[genero] == 1].groupby(by=['user_id'])['playtime_forever'].sum().reset_index()
    if not df_append.empty:
        # Encontrar el jugador con la máxima cantidad de horas jugadas
        max_hours_player = df_append.loc[df_append['playtime_forever'].idxmax(), 'user_id']
    else:
    # Manejar el caso en que la Serie esté vacía
        max_hours_player= 'Player Not Found'
    df_genres.at[index, 'PlayerMH'] = max_hours_player


In [33]:
## AÑO JUGADOR POR CADA JUGADOR

#primero debemos incluir el año en que jugó en una columna de users_items
df_users_items=df_users_items.merge(df_user_item_played_year, on=['item_id','user_id'],how='left')

# Itero sobre las filas de df_genresplayer
for index, row in df_genres.iterrows():
    genero = row['Genre']
    jugador = row['PlayerMH']
    
    # Filtro df_users_items para obtener las horas jugadas del jugador por género y año
    df_append_player = df_users_items[(df_users_items[genero] == 1) & (df_users_items['user_id'] == jugador)].groupby(by=['year'])['playtime_forever'].sum().reset_index()
    df_append_player = df_append_player.sort_values(by='playtime_forever', ascending=False)
    
    # Obtengo los años jugados y horas jugadas
    years_by_player = []
    for j in range(min(3, len(df_append_player))):
        this_year = '{Año: ' + str(int(df_append_player['year'].iloc[j])) + ', Horas: ' + str(int(df_append_player['playtime_forever'].iloc[j])) + '}'
        years_by_player.append(this_year)
    years_by_player=str(years_by_player)

    # Asigno la lista de años jugados y horas jugadas a la columna 'Years Played' en la fila actual
    df_genres.at[index, 'Years Played by player'] = years_by_player
    

In [35]:
#se exporta el dataframe
df_genres.to_csv('Datos_procesados/genres_analysis.csv')

In [38]:
df_steamgames.columns

Index(['publisher', 'genres', 'app_name', 'item_title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'item_id',
       'developer', 'Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing', 'Early Access',
       'Massively Multiplayer', 'Animation & Modeling', 'Video Production',
       'Utilities', 'Web Publishing', 'Education', 'Software Training',
       'Design & Illustration', 'Audio Production', 'Photo Editing',
       'Accounting', 'release_year', 'playtime_forever'],
      dtype='object')

#### df_year_recommendations

In [143]:
#se genera el dataframe que se va a exportar
df_year_recommendations=pd.DataFrame(df_user_reviews['year'].unique())
#se cambia el nombre de la columna
df_year_recommendations.rename(columns={0:'year'}, inplace=True)
#se generan las columnas para las recomendaciones y no recomendaciones
df_year_recommendations['recommend_id_1']=0
df_year_recommendations['recommend_id_2']=0
df_year_recommendations['recommend_id_3']=0
df_year_recommendations['not_recommend_id_1']=0
df_year_recommendations['not_recommend_id_2']=0
df_year_recommendations['not_recommend_id_3']=0

In [162]:
# Para los recomendados
#Filtro el dataframe
df_user_reviews_recommend = df_user_reviews[(df_user_reviews['recommend'] == True) & (df_user_reviews['sentiment_analysis'] >= 1)]
#saco columnas que no sirven
df_user_reviews_recommend.drop(['user_id', 'posted', 'recommend', 'date'], axis=1, inplace=True)
# Agrupo por item_id y year, y el promedio de recommend
df_grouped_recommend = df_user_reviews_recommend.groupby(['item_id', 'year']).mean().reset_index()
# Para los no recomendados
# Filtro el dataframe
df_user_reviews_not_recommend = df_user_reviews[(df_user_reviews['recommend'] == False) & (df_user_reviews['sentiment_analysis'] < 1)]
#saco columnas que no sirven
df_user_reviews_not_recommend.drop(['user_id', 'posted', 'recommend', 'date'], axis=1, inplace=True)
# Agrupo por item_id y year, y el promedio de recommend
df_grouped_not_recommend = df_user_reviews_not_recommend.groupby(['item_id', 'year']).mean().reset_index()

C:\Users\feder\AppData\Local\Temp\ipykernel_16760\3016255433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_reviews_recommend.drop(['user_id', 'posted', 'recommend', 'date'], axis=1, inplace=True)
C:\Users\feder\AppData\Local\Temp\ipykernel_16760\3016255433.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_reviews_not_recommend.drop(['user_id', 'posted', 'recommend', 'date'], axis=1, inplace=True)


In [164]:
df_grouped_not_recommend

,item_id,year,sentiment_analysis
0,10,2015,0.0
1,10180,2013,0.0
2,10180,2014,0.0
3,10180,2016,0.0
4,10220,2014,0.0
...,...,...,...
1481,99810,2015,0.0
1482,99900,2014,0.0
1483,99900,2015,0.0
1484,99900,2016,0.0


In [165]:
#filtro por el año y reordeno los valores
for index,row in df_year_recommendations.iterrows():
    anio=row['year']
    #se verifica que este el año en el dataframe así no arroja errores
    if anio in df_grouped_recommend['year'].unique():
        #se genera un nuevo dataframe con ese año filtrado, ordenado de mayor a menor por la cantidad de 
        mejores3= df_grouped_recommend[df_grouped_recommend['year']==anio].sort_values(by='sentiment_analysis', ascending=False).head(3)
        mejores3.reset_index(inplace=True)
        df_year_recommendations.at[index,'recommend_id_1']=mejores3.at[0,'item_id']
        df_year_recommendations.at[index,'recommend_id_2']=mejores3.at[1,'item_id']
        df_year_recommendations.at[index,'recommend_id_3']=mejores3.at[2,'item_id']
    #se verifica que este el año en el dataframe así no arroja errores
    if anio in df_grouped_not_recommend['year'].unique():
        peores3= df_grouped_not_recommend[df_grouped_not_recommend['year']==anio].sort_values(by='sentiment_analysis', ascending=True).head(3)
        peores3.reset_index(inplace=True)
        df_year_recommendations.at[index,'not_recommend_id_1']=peores3.at[0,'item_id']
        df_year_recommendations.at[index,'not_recommend_id_2']=peores3.at[1,'item_id']
        df_year_recommendations.at[index,'not_recommend_id_3']=peores3.at[2,'item_id']

#se indica el indice y se exporta el archivo
df_year_recommendations.set_index('year')
df_year_recommendations.to_csv('Datos_procesados/year_recommendations.csv')


In [166]:
df_year_recommendations

,year,recommend_id_1,recommend_id_2,recommend_id_3,not_recommend_id_1,not_recommend_id_2,not_recommend_id_3
0,2011,10,56400,45300,105400,107900,18700
1,2014,254260,255260,253350,10180,283660,282440
2,2013,234710,236090,239070,10180,250420,248710
3,2016,46280,458730,436090,10180,370240,369200
4,2015,299260,339120,266510,10,331440,33130
5,2012,10,38400,3483,209100,42920,440
6,2010,10150,300,7200,0,0,0


In [ ]:
'''
Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. 
(reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. 
(reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

'''



#### df_releaseyear_sentiment

In [ ]:
'''
Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios 
que se encuentren categorizados con un análisis de sentimiento.
# Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278} 
'''

# Load

### UsersRecommend

### UsersNotRecommend

In [ ]:
''' '''

def UsersNotRecommend( año : int ):

### sentiment_analysis

In [ ]:
''' '''

def sentiment_analysis( año : int ):

# EDA

# ML

In [ ]:
'''Si es un sistema de recomendación item-item:
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.'''

def recomendacion_juego( id de producto ):

In [ ]:
'''Si es un sistema de recomendación user-item:
 Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.
'''

def recomendacion_usuario( id de usuario ):